# ADTs Gymnasium Env

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from adt_env import AttackDefenseTreeEnv

# Set random seed for reproducibility
np.random.seed(42)

In [2]:
# Load the JSON
with open('envs/adt_nuovo_env.json', 'r') as f:
    env_spec = json.load(f)

In [3]:
# Explore the state space
print("State Space Variables:")
state_df = pd.DataFrame.from_dict(env_spec['state_space'], orient='index')
display(state_df)

State Space Variables:


,type,low,high,description
DataExfiltration,discrete,0,1,Goal achievement state
current_player,discrete,0,1,"Current player (0=attacker, 1=defender)"
AccesstoSensitiveFiles,discrete,0,2,Attacker attribute: AccesstoSensitiveFiles
AccesstoReverseShell,discrete,0,2,Attacker attribute: AccesstoReverseShell
AccesstoMySQL,discrete,0,2,Attacker attribute: AccesstoMySQL
WebReconSuccesful,discrete,0,2,Attacker attribute: WebReconSuccesful
AccesstoExecuteArbitraryCode,discrete,0,2,Attacker attribute: AccesstoExecuteArbitraryCode
UnencryptedFiles,discrete,1,2,Initial system attribute: UnencryptedFiles
SOCKS5ProxyActive,discrete,1,2,Initial system attribute: SOCKS5ProxyActive
MisconfiguredApache,discrete,1,2,Initial system attribute: MisconfiguredApache


In [4]:
# Explore attacker actions
print("Attacker Actions:")
attacker_actions = []
for action_id, action_spec in env_spec['action_space']['attacker']['actions'].items():
    attacker_actions.append({
        'ID': action_id,
        'Name': action_spec['name'],
        'Effect': action_spec['effect'],
        'Cost': action_spec['cost'],
    })

attacker_df = pd.DataFrame(attacker_actions)
display(attacker_df)

Attacker Actions:


,ID,Name,Effect,Cost
0,0,exfiltrateData,DataExfiltration,50
1,1,getLoginData,AccesstoMySQL,10
2,2,bufferOverflow,AccesstoExecuteArbitraryCode,40
3,3,getFiles,AccesstoSensitiveFiles,30
4,4,webRecon,WebReconSuccesful,5
5,5,pathTraversal,AccesstoReverseShell,20


In [5]:
# Explore defender actions
print("Defender Actions:")
defender_actions = []
for action_id, action_spec in env_spec['action_space']['defender']['actions'].items():
    defender_actions.append({
        'ID': action_id,
        'Name': action_spec['name'],
        'Effect': action_spec['effect'],
        'Cost': action_spec['cost']
    })

defender_df = pd.DataFrame(defender_actions)
display(defender_df)

Defender Actions:


,ID,Name,Effect,Cost
0,0,changeCredentials,AccesstoMySQL,200
1,1,changeFilePermissions,AccesstoSensitiveFiles,60
2,2,encryptFile,UnencryptedFiles,150
3,3,deactivateSOCKS5Proxy,SOCKS5ProxyActive,120
4,4,reconfigureApache,MisconfiguredApache,50
5,5,disableCGIScripts,CGIscriptsenabled,45
6,6,updateApache,VulnerableApacheHTTPServerVersion,20


In [6]:
# Create the environment
env = AttackDefenseTreeEnv("envs/adt_nuovo_env.json", render_mode="human")

print(f"Environment loaded successfully!")
print(f"State space: {env.observation_space}")
print(f"Action space: {env.action_space}")
print(f"Goal: {env.goal}")
print(f"Number of attacker actions: {env.num_attacker_actions}")
print(f"Number of defender actions: {env.num_defender_actions}")

Number of attacker actions: 6
Number of defender actions: 7
Environment loaded successfully!
State space: Box([0 0 0 0 0 0 0 1 1 1 1 1 1], [1 1 2 2 2 2 2 2 2 2 2 2 2], (13,), int32)
Action space: Discrete(7)
Goal: DataExfiltration
Number of attacker actions: 6
Number of defender actions: 7


In [7]:
# Reset environment and examine initial state
obs, info = env.reset()
print("Initial observation vector:", obs)
print("Initial info:", info)
print("\nInitial state dictionary:")
for key, value in env.state.items():
    print(f"  {key}: {value}")

Initial observation vector: [0 0 0 0 0 0 0 1 1 1 1 1 1]
Initial info: {'current_player': 'attacker', 'goal': 'DataExfiltration', 'terminal': False}

Initial state dictionary:
  DataExfiltration: 0
  current_player: 0
  AccesstoSensitiveFiles: 0
  AccesstoReverseShell: 0
  AccesstoMySQL: 0
  WebReconSuccesful: 0
  AccesstoExecuteArbitraryCode: 0
  UnencryptedFiles: 1
  SOCKS5ProxyActive: 1
  MisconfiguredApache: 1
  WebserverPubliclyExposed: 1
  CGIscriptsenabled: 1
  VulnerableApacheHTTPServerVersion: 1


In [8]:
# Function to get action details
def get_action_details(env, player, action_id):
    if player == "attacker":
        return env.attacker_actions[str(action_id)]
    else:
        return env.defender_actions[str(action_id)]

# Test available actions for current player
available_actions = env.get_available_actions()
current_player = "attacker" if env.state["current_player"] == 0 else "defender"

print(f"Current player: {current_player}")
print(f"Available actions: {available_actions}")
print("\nAction details:")
for action_id in available_actions:
    action_spec = get_action_details(env, current_player, action_id)
    print(f"  Action {action_id}: {action_spec['name']} (cost: {action_spec['cost']})")

Available actions for attacker: {'0': {'name': 'exfiltrateData', 'preconditions': {'conditions': ['AccesstoMySQL', 'AccesstoExecuteArbitraryCode'], 'logic': 'OR', 'required_values': {'AccesstoMySQL': 1, 'AccesstoExecuteArbitraryCode': 1}}, 'effect': 'DataExfiltration', 'cost': 50, 'refinement': 'disjunctive'}, '1': {'name': 'getLoginData', 'preconditions': {'conditions': ['UnencryptedFiles', 'AccesstoSensitiveFiles'], 'logic': 'AND', 'required_values': {'UnencryptedFiles': 1, 'AccesstoSensitiveFiles': 1}}, 'effect': 'AccesstoMySQL', 'cost': 10, 'refinement': 'conjunctive'}, '2': {'name': 'bufferOverflow', 'preconditions': {'conditions': ['SOCKS5ProxyActive', 'WebReconSuccesful'], 'logic': 'AND', 'required_values': {'SOCKS5ProxyActive': 1, 'WebReconSuccesful': 1}}, 'effect': 'AccesstoExecuteArbitraryCode', 'cost': 40, 'refinement': 'conjunctive'}, '3': {'name': 'getFiles', 'preconditions': {'conditions': ['MisconfiguredApache', 'AccesstoReverseShell'], 'logic': 'AND', 'required_values':

In [9]:
# Test a single action step
if available_actions:
    test_action = available_actions[0]  # Take first available action
    action_spec = get_action_details(env, current_player, test_action)
    
    print(f"Testing action {test_action}: {action_spec['name']}")
    print(f"Before action - {action_spec['effect']}: {env.state[action_spec['effect']]}")
    
    obs, reward, terminated, truncated, info = env.step(test_action)
    
    print(f"After action - {action_spec['effect']}: {env.state[action_spec['effect']]}")
    print(f"Reward: {reward}")
    print(f"Terminated: {terminated}")
    print(f"Action info: {info}")

Testing action 4: webRecon
Before action - WebReconSuccesful: 0
After action - WebReconSuccesful: 1
Reward: -5
Terminated: False
Action info: {'current_player': 'defender', 'action_taken': 'webRecon', 'action_valid': True, 'terminal': False}


In [10]:
# Reset for full game simulation
env.reset()

# Track game history
game_history = []
step = 0
max_steps = 20

print("=== Full Game Simulation ===")
env.render()

while step < max_steps:
    current_player = "attacker" if env.state["current_player"] == 0 else "defender"
    available_actions = env.get_available_actions()
    
    if available_actions:
        # Choose action (random for demo)
        action = np.random.choice(available_actions)
        action_spec = get_action_details(env, current_player, action)
    else:
        # No actions available, choose wait/pass
        if current_player == "attacker":
            action = env.num_attacker_actions - 1
        else:
            action = env.num_defender_actions - 1
        action_spec = {"name": "wait", "cost": 0}
    
    # Execute action
    obs, reward, terminated, truncated, info = env.step(action)
    
    # Record step
    game_history.append({
        'step': step + 1,
        'player': current_player,
        'action': action_spec['name'],
        'reward': reward,
        'terminated': terminated,
        'goal_state': env.state[env.goal]
    })
    
    print(f"\nStep {step + 1}: {current_player} -> {action_spec['name']} (reward: {reward})")
    
    if terminated or truncated:
        print(f"\n=== Game Ended after {step + 1} steps! ===")
        env.render()
        break
    
    step += 1

# Display game history
print("\n=== Game History ===")
history_df = pd.DataFrame(game_history)
display(history_df)

=== Full Game Simulation ===

=== Current Player: Attacker ===
Goal (DataExfiltration): 0
Key Attributes:
  DataExfiltration: 0
  AccesstoSensitiveFiles: 0
  AccesstoReverseShell: 0
  AccesstoMySQL: 0
  WebReconSuccesful: 0
  AccesstoExecuteArbitraryCode: 0
  UnencryptedFiles: 1
  SOCKS5ProxyActive: 1
  MisconfiguredApache: 1
  WebserverPubliclyExposed: 1
  CGIscriptsenabled: 1
  VulnerableApacheHTTPServerVersion: 1
Available actions for attacker: {'0': {'name': 'exfiltrateData', 'preconditions': {'conditions': ['AccesstoMySQL', 'AccesstoExecuteArbitraryCode'], 'logic': 'OR', 'required_values': {'AccesstoMySQL': 1, 'AccesstoExecuteArbitraryCode': 1}}, 'effect': 'DataExfiltration', 'cost': 50, 'refinement': 'disjunctive'}, '1': {'name': 'getLoginData', 'preconditions': {'conditions': ['UnencryptedFiles', 'AccesstoSensitiveFiles'], 'logic': 'AND', 'required_values': {'UnencryptedFiles': 1, 'AccesstoSensitiveFiles': 1}}, 'effect': 'AccesstoMySQL', 'cost': 10, 'refinement': 'conjunctive'},

,step,player,action,reward,terminated,goal_state
0,1,attacker,webRecon,-5,False,0
1,2,defender,changeCredentials,-200,False,0
2,3,attacker,pathTraversal,-20,False,0
3,4,defender,changeFilePermissions,-60,False,0
4,5,attacker,bufferOverflow,-40,False,0
5,6,defender,wait,0,False,0
6,7,attacker,exfiltrateData,-550,True,1
